In [2]:
import numpy as np
import pandas as pd
import eurostat
import pycountry
import seaborn as sns

In [3]:
#toc_df = eurostat.get_toc_df()
#toc_df
#info = eurostat.subset_toc_df(toc_df, 'demo_r_mwk_10')
#info
df = eurostat.get_data_df('demo_r_mwk_20')
df.columns = df.columns.str.replace('\\', '_',regex=False)
df = df[df.age != 'UNK']
df.pop = eurostat.get_data_df('demo_pjangroup')
df.pop

,unit,sex,age,geo\time,2021,2020,2019,2018,2017,2016,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,NR,F,TOTAL,AD,NaN,NaN,37388.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NR,F,TOTAL,AL,NaN,1425342.0,1432833.0,1431715.0,1423050.0,1417141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NR,F,TOTAL,AM,NaN,1562689.0,1563538.0,1564533.0,1567380.0,1569535.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NR,F,TOTAL,AT,NaN,4522292.0,4501742.0,4483749.0,4460424.0,4427918.0,...,3932691.0,3922359.0,3899799.0,3876559.0,3857760.0,3836415.0,3814191.0,3794130.0,3773097.0,3757167.0
4,NR,F,TOTAL,AZ,NaN,5039100.0,4999053.0,4960058.0,4918771.0,4870002.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3766,NR,T,Y_LT5,SM,NaN,NaN,NaN,1445.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3767,NR,T,Y_LT5,TR,NaN,6345136.0,6484986.0,6481849.0,6459295.0,6381516.0,...,5018896.0,4891997.0,4768664.0,4648776.0,4532233.0,4454837.0,4379570.0,4306379.0,4235200.0,4165977.0
3768,NR,T,Y_LT5,UA,NaN,1805636.0,1960529.0,2097973.0,2224075.0,2301004.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3769,NR,T,Y_LT5,UK,NaN,NaN,3885007.0,3948754.0,3987530.0,4020705.0,...,4752900.0,4790000.0,4795700.0,4767400.0,4700000.0,4600000.0,4500000.0,4300000.0,4200000.0,4100000.0


In [4]:
uniqueValues = df.pop['age'].unique()
uniqueValues

array(['TOTAL', 'UNK', 'Y10-14', 'Y15-19', 'Y20-24', 'Y25-29', 'Y30-34',
       'Y35-39', 'Y40-44', 'Y45-49', 'Y5-9', 'Y50-54', 'Y55-59', 'Y60-64',
       'Y65-69', 'Y70-74', 'Y75-79', 'Y80-84', 'Y_GE75', 'Y_GE80',
       'Y_GE85', 'Y_LT5'], dtype=object)

In [3]:
df['age'].replace(['Y20-39', 'Y40-59', 'Y60-79', 'Y_GE80', 'Y_LT20'],['20-39', '40-59', '60-79', '80+', '0-20'], inplace=True)
df2=pd.melt(df,id_vars=['age','sex','unit','geo_time'],var_name='yearweek', value_name='deaths')
df2

,age,sex,unit,geo_time,yearweek,deaths
0,TOTAL,F,NR,AD,2021W50,NaN
1,TOTAL,F,NR,AL,2021W50,NaN
2,TOTAL,F,NR,AM,2021W50,NaN
3,TOTAL,F,NR,AT,2021W50,NaN
4,TOTAL,F,NR,BE,2021W50,NaN
...,...,...,...,...,...,...
755209,0-20,T,NR,RS,2000W01,31.0
755210,0-20,T,NR,SE,2000W01,15.0
755211,0-20,T,NR,SI,2000W01,6.0
755212,0-20,T,NR,SK,2000W01,18.0


In [4]:
df_clean = df2.drop(columns = ['unit'])
df_clean[['year','week']] = df_clean.yearweek.str.split("W",expand=True)
df_clean['week'] = df_clean.week.str.extract('(\d+)')
df_clean['year'] = df_clean.year.str.extract('(\d+)')
df_clean['geo_time'] = df_clean['geo_time'].replace("UK", "GB")
df_clean['geo_time'] = df_clean['geo_time'].replace("EL", "GR")
countries = {}
for country in pycountry.countries:
    countries[country.alpha_2] = country.name
df_clean['country'] = [countries.get(country, 'Unknown code') for country in df_clean.geo_time]
df_clean = df_clean[['country','sex','age','yearweek','year','week','deaths']]
#remove this line to get the full time period
df_clean['covid_year']=df_clean['year'] >= '2020'
df_clean.loc[df_clean['covid_year'] == False, 'covid_year'] = '2000-2019 +/- SD'
df_clean.loc[df_clean['covid_year'] == True, 'covid_year'] = df_clean['year']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

df_clean.iloc[0:10]

,country,sex,age,yearweek,year,week,deaths,covid_year
0,Andorra,F,TOTAL,2021W50,2021,50,NaN,2021
1,Albania,F,TOTAL,2021W50,2021,50,NaN,2021
2,Armenia,F,TOTAL,2021W50,2021,50,NaN,2021
3,Austria,F,TOTAL,2021W50,2021,50,NaN,2021
4,Belgium,F,TOTAL,2021W50,2021,50,NaN,2021
5,Bulgaria,F,TOTAL,2021W50,2021,50,NaN,2021
6,Switzerland,F,TOTAL,2021W50,2021,50,NaN,2021
7,Cyprus,F,TOTAL,2021W50,2021,50,NaN,2021
8,Czechia,F,TOTAL,2021W50,2021,50,NaN,2021
9,Germany,F,TOTAL,2021W50,2021,50,NaN,2021


In [ ]:
g = sns.FacetGrid(df_clean.query("sex == 'T'"), col="age", hue="covid_year", row='country', aspect=2,sharey=False)
g.map(sns.lineplot, 'week', 'deaths', alpha=.7, estimator='mean', ci='sd')
g.set(xlabel="month", ylabel = "deaths per week", xticks=np.arange(1, 53,(53/12) ), xticklabels=months)
g.add_legend(title = '')
for suffix in 'png svg'.split():
    g.savefig('by_country_age.'+suffix, dpi=200, bbox_inches='tight', facecolor='white')

In [ ]:
g = sns.FacetGrid(df_clean.query("age == 'TOTAL'"), col="sex", hue="covid_year", row='country', aspect=2,sharey=False)
g.map(sns.lineplot, 'week', 'deaths', alpha=.7, estimator='mean', ci='sd')
g.set(xlabel="month", ylabel = "deaths per week", xticks=np.arange(1, 53,(53/12) ), xticklabels=months)
g.add_legend(title = '')
for suffix in 'png svg'.split():
    g.savefig('by_country_sex.'+suffix, dpi=200, bbox_inches='tight', facecolor='white')

In [9]:
df_total=df_clean.query("age == 'TOTAL' & sex == 'T'").groupby(['yearweek', 'year', 'week', 'covid_year'], as_index=False)['deaths'].sum().reset_index(drop=True)
df_total

,yearweek,year,week,covid_year,deaths
0,2000W01,2000,01,2000-2019 +/- SD,67886.0
1,2000W02,2000,02,2000-2019 +/- SD,68745.0
2,2000W03,2000,03,2000-2019 +/- SD,69668.0
3,2000W04,2000,04,2000-2019 +/- SD,70023.0
4,2000W05,2000,05,2000-2019 +/- SD,68629.0
...,...,...,...,...,...
1141,2021W46,2021,46,2021,83868.0
1142,2021W47,2021,47,2021,71558.0
1143,2021W48,2021,48,2021,66365.0
1144,2021W49,2021,49,2021,43261.0


In [8]:
g = sns.FacetGrid(df_total, hue="covid_year", aspect=2,sharey=False)
g.map(sns.lineplot, 'week', 'deaths', alpha=.7, estimator='mean', ci='sd')
g.set(xlabel="month", ylabel = "deaths per week", xticks=np.arange(1, 53,(53/12) ), xticklabels=months)
g.add_legend(title = '')
for suffix in 'png svg'.split():
    g.savefig('total.'+suffix, dpi=200, bbox_inches='tight', facecolor='white')

SyntaxError: positional argument follows keyword argument (611397274.py, line 1)